# Social Smart Meter

## Imports

In [1]:
import time
from datetime import datetime

from config import instagram_config, twitter_config, facebook_config

## Input parameters

In [2]:
# Amsterdam neighborhoods
amsterdam = {
    'Centrum': {'latitude': '52.372374', 'longitude': '4.898844', 'radius': '1500'},
    'Noord': {'latitude': '52.395940', 'longitude': '4.924071', 'radius': '2000'},
    'Oost': {'latitude': '', 'longitude': '', 'radius': ''},
    'Zuidoost': {'latitude': '52.304499', 'longitude': '4.971431', 'radius': '2000'},
    'Zuid': {'latitude': '52.342684', 'longitude': '4.885141', 'radius': '2000'},
    'West': {'latitude': '52.388177', 'longitude': '4.862139', 'radius': '1500'},
    'Nieuw-West': {'latitude': '52.360300', 'longitude': '4.810984', 'radius': '3000'},
    'Westpoort': {'latitude': '52.403051', 'longitude': '4.826776', 'radius': '2000'}
}

In [3]:
# Choose a neighborhood
neighborhood = amsterdam['Centrum']

### Instagram

In [42]:
# Coordinates
latitude = neighborhood['latitude']
longitude = neighborhood['longitude']

# Radius in meters
distance = neighborhood['radius']

# Number of posts to retrieve (max = 100)
count = 100

# Start date
min_dt = datetime(2018, 5, 20, 17, 0, 0)
min_timestamp = time.mktime(min_dt.timetuple())

# End date
max_dt = datetime(2018, 5, 20, 18, 0, 0)
max_timestamp = time.mktime(max_dt.timetuple())

In [43]:
# Store parameters in object
params = {
    'lat': latitude,
    'lng': longitude,
    'distance': distance,  # radius of requested area
    'min_timestamp': str(min_timestamp),  # start date
    'max_timestamp': str(max_timestamp),  # end date
    'count': count,  # number of posts(100 max)
    'access_token': instagram_config['access_token']  # your access token
}

### Twitter

In [6]:
# TODO

## Data collection

In [4]:
from data_utils import connect_to_db

In [5]:
# Choose a database to store the data
db_name = 'thesis'

# Connect to the database
db = connect_to_db(db_name)

### Instagram

In [45]:
from instagram_utils import collect_instagram_data

In [46]:
# Get JSON response via Instagram API, store it in the database, and return the corresponding ObjectId
object_id = collect_instagram_data(params, db)

### Twitter

In [11]:
# TODO

## Data processing 

In [47]:
from data_utils import process_data

In [48]:
# Specify whether to save the images or not
save_images = True

# Process data to store relevant attributes in output collection
process_data(object_id, db, save_images)

## Data enrichment

In [6]:
from bson.objectid import ObjectId

In [7]:
# In case the document object differs from the one above, copy value from database
object_id = ObjectId("5b034a1c7304aaa054bdcd71")

### Enrich text

Determine the language of the text using PyPi's language detection library ported from Google's language-detection.

In [16]:
# TODO

### Enrich place

Match Instagram place to Facebook place and determine this place's place topics (or place categories).

In [49]:
from facebook_utils import connect_to_facebook_api

In [65]:
# Request short-term access token and paste it below
short_term_access_token = 'EAACEdEose0cBAKmcPzUFRfAPl6CW0msqKwWoereWHlmZBwGZB08MNEZA5qjlwXFU9v05DJgQiG4dQWIrlJoZBoZBdoQdzVW1DPjxSRrkvKK9ELmCVAZAzVNJ8mm6ovuafjMzxIS5ULD3aYqjqM7JwLadqF4ZB4JQlOQ0d9FNEonZBpixjvnCztpOksua9VZBH1T7ZANvgmsv2syQZDZD'

# Update facebook config object
facebook_config['access_token'] = short_term_access_token

In [66]:
# Choose which (access) token type to use:
# -'long_term_token' for the long-term access token
# - 'user_token' for the short-term user token
# - anything else (e.g., '') for the short-term access token
token_type = ''

# Connect to Facebook API
graph = connect_to_facebook_api(token_type)

Compare places from current and previous posts to determine the distance traveled between these posts.

### Enrich user profile

Match Instagram user profile to Twitter one to find a user's home town.

In [20]:
# TODO

### Enrich the data

In [67]:
from data_utils import enrich_data

In [68]:
# Determine which enrichments to apply to the data
enrichments = {
    'text_language': False,
    'place_categories': True,
    'place_distance': False,
    'user_home': False
}

In [69]:
# Perform the enrichments
enrich_data(enrichments, object_id, db, graph)

## Data classification

### Update dictionaries

In [8]:
from dictionary_utils import merge_dictionaries

In [9]:
merge_dictionaries()

#### Manual updates

In [10]:
from dictionaries import merged_dictionaries
# merged_dictionaries['leisure'].append('diving')
# merged_dictionaries['mobility'].append('parked')
merged_dictionaries['food_consumption'].append('coffee')
merged_dictionaries['leisure'].append('coffee')

### Text processing models

In [27]:
# To be implemented..

# Directory to save trained model
# ner_model_dir = './models/ner/'

# Train NER model
# train_model(ner_model_dir)

### Image processing models

#### Mask R-CNN model

In [28]:
from image_utils import get_mrcnn_model

In [29]:
# Get pre-trained Mask C-RNN model
mrcnn_model, mrcnn_dataset = get_mrcnn_model()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           coco
NUM_CLASSES                    81
POOL_SIZE                      7
POST_

#### TensorFlow model 

Any model exported using the export_inference_graph.py tool can be loaded here simply by changing PATH_TO_CKPT to point to a new .pb file. By default we use an "SSD with Mobilenet" model here. See the detection model zoo for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [30]:
from image_utils import get_tf_model

In [31]:
tf_detection_graph, tf_category_index = get_tf_model()

#### Use other pre-trained models

In [32]:
# import pickle

# with open('./models/X-152-32x8d-IN5k.pkl', 'rb') as f:
#     model = pickle.load(f)

In [33]:
# import pandas

# path='./models/X-152-32x8d-IN5k.pkl'

# weights = pandas.read_pickle(path, compression='infer')

In [34]:
# Release of new weight files:

# https://github.com/fchollet/deep-learning-models/releases/tag/v0.8

### Classify data

In [15]:
from data_utils import classify_data

In [16]:
# Determine which enrichments to apply to the data
classification = {
    'text': False,
    'image': True,
    'place': True
}

In [17]:
# If you don't need to use the models:
mrcnn_model = None
mrcnn_dataset = None
tf_detection_graph = None
tf_category_index = None

# Store models in object and specify whether to run the model or not
models = {
    'mrcnn': {'run': True, 'model': mrcnn_model, 'dataset': mrcnn_dataset},
    'tf': {'run': True, 'detection_graph': tf_detection_graph, 'category_index': tf_category_index}
}

In [18]:
classify_data(classification, models, object_id, db)

## Data visualization

In [15]:
# TODO

# Playground

In [40]:
import os
import skimage.io

from mrcnn import model as modellib
from mrcnn import utils
from mrcnn import visualize

from coco.coco import CocoConfig
from coco.coco import CocoDataset

from tensor_flow_object_detection import load_model, run_model_on_single_image

In [41]:
post_id = '1782971715335144481_198080822'
image_url = 'https://scontent.cdninstagram.com/vp/814dae94951bd82a92dcc4793aa03648/5B90D2B0/t51.2885-15/s640x640/sh0.08/e35/32237915_2121819221383199_6148507372286377984_n.jpg'
image = skimage.io.imread(image_url)

In [42]:
image_path = '../data/instagram/images/input/{}.jpg'.format(post_id)

print(image_path)

../data/instagram/images/input/1782971715335144481_198080822.jpg


In [7]:
# Define what model to download
model_name = 'faster_rcnn_inception_resnet_v2_atrous_oid_2018_01_28'    # Alternative: 'ssd_mobilenet_v1_coco_2017_11_17'
model_file = model_name + '.tar.gz'
download_base = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
path_to_ckpt = model_name + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
path_to_labels = 'object_detection/data/oid_bbox_trainable_label_map.pbtxt' # Alternative: 'mscoco_label_map.pbtxt'

num_classes = 545   # Alternative: 90

# Create params object
tf_params = {
    'model_name': model_name,
    'model_file': model_file,
    'download_base': download_base,
    'path_to_ckpt': path_to_ckpt,
    'path_to_labels': path_to_labels,
    'num_classes': num_classes
}

In [8]:
# Load Tensorflow model
tf_detection_graph, tf_category_index = load_model(tf_params)

In [44]:
# Run model on image
output_dict = run_model_on_single_image(image, post_id, tf_detection_graph, tf_category_index)

<Figure size 864x864 with 0 Axes>

In [45]:
print(output_dict)

{'num_detections': 3, 'detection_boxes': array([[0.06364858, 0.06219246, 0.88002086, 0.7775232 ],
       [0.11871229, 0.03846986, 0.51375073, 0.39425084],
       [0.        , 0.00218953, 0.7550753 , 0.6042471 ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0. 